In [19]:
from functions.satellites_ephemeris import SatelliteEphemeris

filename = 'functions/ephemerides/OneWeb_Sat_Data.xlsx'
ephemeris = SatelliteEphemeris(filename=filename)
ephemeris.load_data()
debris_data = ephemeris.get_debris()

Custom database loaded.


In [28]:
t1 = 8487.72551337956
id1 = '0'

t2 = 8492.72551337956
id2 = '200'

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

In [29]:
from functions.MRPLP_J2_analytic import MultiRevolutionPerturbedLambertSolver
from functions.expansion_perturbed_lambert import ExpansionPerturbedLambert
import numpy as np

rr1=cart1[0:3]
vv1=cart1[3:]
rr2=cart2[0:3]
vv2=cart2[3:]
vv1g = vv1

tof = ( t2 - t1 ) * 86400

mu = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

MRPLPsolver = MultiRevolutionPerturbedLambertSolver() # MRPLP solver
EXPpertLamb = ExpansionPerturbedLambert() # Expansion of perturbed Lambert

# set the parameters for the MRPLP solver
order = 7
parameters = MRPLPsolver.mrplp_J2_analytic_parameters(rr1, rr2, tof, vv1g, mu, Re, J2,
                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, scl=1.0e-3, itermax=200 )

# solve the MRPLP
output = MRPLPsolver.mrplp_J2_analytic(parameters)

# extract the solution
vv1Sol = output.vv1Sol
vv2Sol = output.vv2Sol
rr2DA = output.rr2DA

# compute the DV
dvv1 = vv1Sol - vv1
dvv2 = vv2 - vv2Sol
dv1 = np.linalg.norm( dvv1 )
dv2 = np.linalg.norm( dvv2 )
dvtot = dv1 + dv2

# print the output
print(f"-------------------------------------------------------")
print(f"                 OUTPUT SUMMARY")
print(f"-------------------------------------------------------")
print(f"Order of the expansion  : {order}")
print(f"Success                 : {output.success}")
print(f"Elapsed time            : {output.elapsed_time} seconds")
print(f"Final pos. error (norm) : {np.linalg.norm( output.rr2DA - rr2 )} m")
print(f"-------------------------------------------------------")
print(f"Delta_v1                : {dv1} m/s")
print(f"Delta_v2                : {dv2} m/s")
print(f"Delta_vtot              : {dvtot} m/s")
print(f"-------------------------------------------------------")


-------------------------------------------------------
                 OUTPUT SUMMARY
-------------------------------------------------------
Order of the expansion  : 7
Success                 : False
Elapsed time            : 9.923449754714966 seconds
Final pos. error (norm) : nan m
-------------------------------------------------------
Delta_v1                : nan m/s
Delta_v2                : nan m/s
Delta_vtot              : nan m/s
-------------------------------------------------------


In [82]:
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2

t1 = 8487.0
id1 = '0'
id2 = '249'

########## START: define the DV options ##########
dv_tol_an = 1500
dv_tol = dv_tol_an - 500
########## END: define the DV options ##########

########## START: define the TOF options ##########
start_tof = 5.0 # days
end_tof = 30.0 # days
grid_size = 2.0 # days
tof_list = np.arange(start_tof, end_tof + grid_size, grid_size)
########## END: define the TOF options ##########

for tof in tof_list:

    t2 = t1 + tof
    tof_secs = tof * 86400.0

    cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
    cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )
    vv1g = cart1[3:]

    # maybe first do an approximate DV computation, and then use the MRPLP
    paramsAnalytical = AnalyticDVApproxJ2.approx_DV_parameters( cart1[0:3], cart1[3:],
                                                                cart2[0:3], cart2[3:],
                                                                t1, t2,
                                                                mu, Re, J2)
    delta_v = AnalyticDVApproxJ2.approx_DV(paramsAnalytical)

    if delta_v <= dv_tol_an:

        # set the parameters for the MRPLP solver
        order = 7
        parameters = MRPLPsolver.mrplp_J2_analytic_parameters(cart1[0:3], cart2[0:3], tof_secs, vv1g, mu, Re, J2,
                                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, 
                                                            scl=1.0e-3, itermax=150 )
        
        # solve the MRPLP
        output = MRPLPsolver.mrplp_J2_analytic(parameters)
        
        # compute the DV
        dvv1 = output.vv1Sol - cart1[3:]
        dvv2 = cart2[3:] - output.vv2Sol
        dv1 = np.linalg.norm( dvv1 )
        dv2 = np.linalg.norm( dvv2 )
        dvtot = dv1 + dv2

        print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {delta_v} m/s - approx. DV")
        if np.isnan(dvtot):
            print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - NaN")
        else:
            if dvtot < dv_tol:
                print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, DV: {dvtot} m/s")
            else:
                print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION")
    
    else:
        print(f"(ID1, t1)=({id1, t1})--> ID2=({id2,t2}), tof: {tof} days, NO SOLUTION - approx. DV: {delta_v} m/s")


(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8492.0)), tof: 5.0 days, DV: 5.269684101075631 m/s - approx. DV
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8492.0)), tof: 5.0 days, DV: 35.20379226413945 m/s
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8494.0)), tof: 7.0 days, DV: 5.157472019331834 m/s - approx. DV
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8494.0)), tof: 7.0 days, DV: 45.143467572856146 m/s
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8496.0)), tof: 9.0 days, DV: 4.978293051813548 m/s - approx. DV
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8496.0)), tof: 9.0 days, DV: 28.579521111078428 m/s
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8498.0)), tof: 11.0 days, DV: 4.845222446836721 m/s - approx. DV
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8498.0)), tof: 11.0 days, DV: 19.38995171049234 m/s
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8500.0)), tof: 13.0 days, DV: 4.663101607920961 m/s - approx. DV
(ID1, t1)=(('0', 8487.0))--> ID2=(('249', 8500.0)), tof: 13.0 days, DV: 23.577106449345862 m/

In [16]:
# propagate and obtain the STM
states, vecttof, STM = (
    MultiRevolutionPerturbedLambertSolver.propagatePerturbedKeplerSTM(
        rr1, vv1Sol, tof, 1000.0, parameters
    )
)

In [17]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Extract x, y, z components
x = states[:, 0]
y = states[:, 1]
z = states[:, 2]

# Create interactive 3D plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(width=2),
    name='trajectory'
)])

# Add initial and final points
fig.add_trace(go.Scatter3d(
    x=[x[0]],
    y=[y[0]],
    z=[z[0]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[1]],
    y=[y[1]],
    z=[z[1]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[2]],
    y=[y[2]],
    z=[z[2]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[3]],
    y=[y[3]],
    z=[z[3]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[-1]],
    y=[y[-1]],
    z=[z[-1]],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Final Point'
))

# Labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()

In [48]:
# analytic computation of DV
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2
import numpy as np


mue = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

id1 = '0'
id2 = '302'
t1 = 8487.0
t2  = t1 + 10.0

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

# Initialize parameters
parameters = AnalyticDVApproxJ2.approx_DV_parameters(
    rr1=cart1[0:3],
    vv1=cart1[3:],
    rr2=cart2[0:3],
    vv2=cart2[3:],
    t1=t1,
    t2=t2,
    mu=mue,
    rE=Re,
    J2=J2
)

delta_v = AnalyticDVApproxJ2.approx_DV(parameters)
delta_v

10338.926437805067